In [43]:
import tarantool
PORT = 3303 # check running container
connection = tarantool.connect(host="localhost", port=PORT, user='admin', password='pass')


In [21]:
'''Команды в консоли тарантула (запущенного в докере)
box.schema.space.create('tester2')

box.space.tester2:format({
    { name = 'id', type = 'unsigned' },
    { name = 'band_name', type = 'string' },
    { name = 'year', type = 'unsigned' }
})

-- Create a primary index --
box.space.tester2:create_index('primary', { parts = { 'id' } })

box.space.tester2:create_index('name', { parts = { 'band_name' } })
'''
space_name = 'tester2'

In [2]:
conn_space = connection.space(space_name)
res = conn_space.select()
res

- [2, 'Adams']
- [3, 'Adams']

In [15]:
res = connection.select(space_name, 0)
res

In [14]:
conn_space.insert((1, 'ABBA', 1972))

- [1, 'ABBA', 1972]

In [13]:
conn_space

In [24]:
res = conn_space.select('ABBA', index=1) # if index 1 already created
res

- [1, 'ABBA', 1972]

In [23]:
def select_by_name(name='ABBA'):
    res = conn_space.select(name, index='name')
    return res

print(select_by_name())

- [1, 'ABBA', 1972]


In [25]:
conn_space.upsert((1, 'ABBA', 0), [('=', 2, 16)]) # 2 - index from 0
res = select_by_name()
res

- [1, 'ABBA', 16]

In [26]:
conn_space.update(1, [('=', 1, 'ABBA'), ('+', 2, 1)]) # updated 2 fields

- [1, 'ABBA', 17]

In [27]:
# Теперь с помощью функции replace мы полностью заменим кортеж 
# с совпадающим первичным ключом. Если кортежа с указанным первичным ключом не существует, 
# то эта операция ни к чему не приведет.

conn_space.replace((4, 'New band', 2015))
res = select_by_name('New band')
res


- [4, 'New band', 2015]

In [6]:
connection.execute('select * from customers')

- [1, 'James Bond']
- [2, 'James Bond']
- [3, 'James Bond']

In [8]:
res = connection.space('customers').select()
res

- [2, 'Adams']
- [3, 'Adams']

In [25]:
res = connection.space('fff').select()
res

In [32]:
'''
box.schema.space.create('tester3')
box.space.tester3:create_index('primary')
and
box.execute([[CREATE TABLE tester3 (id INTEGER PRIMARY KEY AUTOINCREMENT, name text);]]) ----> it creates TESTER3 with capital letters, even when written with small
'''
# 
# connection.space('tester3').insert((1, 'ABBA', 1972))
# res1 = connection.space('tester3').select()
# print(res1)


connection.execute('insert into tester8 (name) values (?);', ("My_Val1",))  # ---->-  it thinks name of the table is capital-letter TESTER8  even when written with small
res2 = connection.execute('select * from tester8') # CAPITALL Letters
print("None" if not res2 else res2)


- [1, 'My_Val1']
- [2, 'My_Val1']
- [3, 'My_Val1']
- [4, 'My_Val1']
- [5, 'My_Val1']


In [51]:
'''for tarantool in docker version 3
sudo docker exec -it t3 console
sudo docker exec -it t4 console
Команды в консоли тарантула (запущенного в докере)

box.cfg {
        memtx_memory = 1 * 1000 * 1024 * 1024,
        listen = 3301,
        checkpoint_count  = 3,
        checkpoint_interval = 3600,
        read_only  = false 
}


box.schema.space.create('APP')

box.space.APP:format({
    { name = 'ID', type = 'uuid' },
    { name = 'BAND_NAME', type = 'string' },
    { name = 'YEAR', type = 'unsigned' }
})

box.space.APP:create_index("primary", {parts={{field = 1, type = 'uuid'}}})

box.space.tester2:create_index('name', { parts = { 'band_name' } })
'''
import uuid


connection.execute('insert into APP (id, band_name, year) values (?, ?, ?);', (uuid.uuid4(), "My_Val1", 20))  # ---->-  it thinks name of the table is capital-letter TESTER8  even when written with small
res2 = connection.execute('select * from SEQSCAN  APP limit 100') # CAPITALL Letters
print("None" if not res2 else res2)

- [UUID('92b72308-b94f-4c13-b895-401925f4623f'), 'My_Val1', 20]
- [UUID('9fa96f35-e3d3-4cff-ac7f-0210813e5d71'), 'My_Val1', 20]
- [UUID('b024b753-c8ca-4612-a03a-34e5d17a7fb6'), 'My_Val1', 20]
- [UUID('e083f994-0a60-43d9-9e26-fe79100220bd'), 'My_Val1', 20]
